In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Пример обращения к поиску (Search):
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

# Выполните поиск по слову "sunflowers". 
# Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.

In [5]:
res

{'total': 85,
 'objectIDs': [436524,
  484935,
  437112,
  210191,
  431264,
  397949,
  656530,
  480725,
  486590,
  375281,
  485308,
  705155,
  11922,
  2032,
  816522,
  343052,
  20141,
  208554,
  2019,
  403496,
  360837,
  207869,
  437115,
  707887,
  400581,
  423400,
  223828,
  682927,
  57922,
  649813,
  79739,
  193938,
  339347,
  208218,
  437984,
  202228,
  436534,
  704667,
  696949,
  761604,
  437329,
  436252,
  436580,
  363282,
  190739,
  207753,
  437526,
  347980,
  822570,
  487043,
  203893,
  367434,
  423237,
  839296,
  367817,
  13644,
  820022,
  648617,
  761731,
  814626,
  207967,
  626832,
  262430,
  262428,
  262423,
  707697,
  205386,
  262378,
  426804,
  664519,
  436121,
  436529,
  633526,
  747779,
  369073,
  436144,
  852414,
  427872,
  436530,
  436535,
  437980,
  724844,
  834585,
  834580,
  834765]}

In [19]:
df = pd.DataFrame(res)
df['objectIDs']

0     436524
1     484935
2     437112
3     210191
4     431264
       ...  
80    437980
81    724844
82    834585
83    834580
84    834765
Name: objectIDs, Length: 85, dtype: int64

In [37]:
# Step 1
list = [2032, 436524, 437329, 16822570, 20149]
df['check'] = df.objectIDs.isin(list)
df.query('check == True')

,total,objectIDs,check
0,85,436524,True
13,85,2032,True
40,85,437329,True


In [41]:
# Step 3
# Пример текста запроса для получения информации об одном объекте (Object):
# r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID')
# obj_test = r_test.json()

# Выполните запрос для объекта 437980. Для проверки соотнесите полученные параметры с их значениями:

r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()
obj_test

{'objectID': 437980,
 'isHighlight': False,
 'accessionNumber': '49.30',
 'accessionYear': '1949',
 'isPublicDomain': True,
 'primaryImage': 'https://images.metmuseum.org/CRDImages/ep/original/DP130999.jpg',
 'primaryImageSmall': 'https://images.metmuseum.org/CRDImages/ep/web-large/DP130999.jpg',
 'additionalImages': [],
 'constituents': [{'constituentID': 161947,
   'role': 'Artist',
   'name': 'Vincent van Gogh',
   'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500115588',
   'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q5582',
   'gender': ''}],
 'department': 'European Paintings',
 'objectName': 'Painting',
 'title': 'Cypresses',
 'culture': '',
 'period': '',
 'dynasty': '',
 'reign': '',
 'portfolio': '',
 'artistRole': 'Artist',
 'artistPrefix': '',
 'artistDisplayName': 'Vincent van Gogh',
 'artistDisplayBio': 'Dutch, Zundert 1853–1890 Auvers-sur-Oise',
 'artistSuffix': '',
 'artistAlphaSort': 'Gogh, Vincent van',
 'artistNationality': 'Dutch',
 'artistB

In [42]:
# создать датафрейм , сначала посмотреть поля на примере одного jsona, а затем прописать их по типу ключ-значение
obj_df = pd.DataFrame({'objectID' : obj_test['objectID'],
                       'title' : [obj_test['title']],
                       'artistDisplayName' : [obj_test['artistDisplayName']], 
                        'department': [obj_test['department']], 
                        'objectBeginDate' : obj_test['objectBeginDate'],  
                        'objectEndDate' : obj_test['objectEndDate'], 
                        'period': [obj_test['period']], 
                        'objectName': [obj_test['objectName']], 
                        'culture' : [obj_test['culture']]})

obj_df

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,


In [48]:
# алгоритм такой: считываем файл с помощью запроса(obj_tmp), 
#                 создаем датафрейм из словаря типа ключ - значение( например для 'culture' это будет obj_tmp['culture']), 
#                 присоединяем к df и так далее по циклу
#  цикл try/except так как есть "битые" файлы

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()
sunflower_ids = res['objectIDs']
full_df = pd.DataFrame()
for sunflower_ids in sunflower_ids:
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{sunflower_ids}')
    obj_test = r_test.json()
    try:
        obj_df = pd.DataFrame({'objectID' : obj_test['objectID'],
                       'title' : [obj_test['title']],
                       'artistDisplayName' : [obj_test['artistDisplayName']], 
                        'department': [obj_test['department']], 
                        'objectBeginDate' : obj_test['objectBeginDate'],  
                        'objectEndDate' : obj_test['objectEndDate'], 
                        'period': [obj_test['period']], 
                        'objectName': [obj_test['objectName']], 
                        'culture' : [obj_test['culture']]})
        full_df = full_df.append(obj_df)
    except:
        pass
full_df

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,Painting,
0,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,Painting,
0,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,Painting,
0,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,Vase,
0,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,Print,
...,...,...,...,...,...,...,...,...,...
0,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,
0,724844,Design for Embroidery with Two Large Garlands ...,Italian School,Drawings and Prints,1845,1905,,Drawing Ornament & Architecture,
0,834585,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,
0,834580,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,


In [49]:
full_df.query("culture == 'Japan'").title.value_counts()

Incense box    1
Name: title, dtype: int64

In [8]:
# Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. 
# Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()

# First 1000 values
cats_id = cats['objectIDs'][:1000]
len(cats_id)

1000

In [11]:
data_with_cats = pd.DataFrame()

#  цикл try/except так как есть "битые" файлы
for id in cats_id:
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{id}')
    object_test = r_test.json()
    try:
        object_df = pd.DataFrame({'objectID' : object_test['objectID'],
                       'title' : [object_test['title']],
                       'artistDisplayName' : [object_test['artistDisplayName']],
                        'period': [object_test['period']], 
                        'objectName': [object_test['objectName']], 
                        'culture' : [object_test['culture']]})
        data_with_cats = data_with_cats.append(object_df)
    except:
        pass
data_with_cats

,objectID,title,artistDisplayName,period,objectName,culture
0,49698,Cat,Zhang Yuguang,,Folding fan mounted as an album leaf,China
0,49470,Cat,Wang Li,Qing dynasty (1644–1911),Hanging scroll,China
0,53222,Musk Cat,Uto Gyoshi,Muromachi period (1392–1573),Hanging scroll,Japan
0,36221,Cat,Wang Yun,,Hanging scroll,China
0,60873,Burial Crown,,Song dynasty (960–1279) or later,Burial crown,China
...,...,...,...,...,...,...
0,63038,"Dish with Sacred Jewels, Plum, Pine, and Bamboo",,Edo period (1615–1868),Dish,Japan
0,63040,Octagonal bowl,,Edo period (1615–1868),Octagonal bowl,Japan
0,63047,Water Jar (Mizusashi) with Geometric Patterns,,late Edo (1615–1868)-early Meiji period (1868–...,Covered water vessel,Japan
0,63034,Ninsei-style Serving Container (Mukōzuke),,Edo period (1615–1868),Cup,Japan


In [12]:
data_with_cats.period.value_counts()

Edo period (1615–1868)                                 434
                                                        96
Ming dynasty (1368–1644)                                50
Qing dynasty (1644–1911)                                49
Muromachi period (1392–1573)                            32
                                                      ... 
Shunga period                                            1
Western Jin dynasty (265–316)                            1
Sui dynasty (581–618)                                    1
late Edo (1615–1868)-early Meiji period (1868–1912)      1
Qing dynasty (1644–1911), Kangxi period (1662–1722)      1
Name: period, Length: 119, dtype: int64